In [22]:
import bz2
import os
import tarfile
import json
from datetime import datetime
import pandas as pd
from unidecode import unidecode
import numpy as np
from fuzzywuzzy import fuzz

In [23]:
#import data
fbref_df = pd.read_csv("clean_data/event_data.csv")
betfair_df = pd.read_csv("betfairhistoricalprices/goalscorer_price_data.csv")

In [24]:
#make deep copy of fbref data
event_df = fbref_df.copy(deep=True)

In [25]:
#helper functions
def transform_dict(input_dict):
    output_dict = {}
    for key, value_list in input_dict.items():
        for value in value_list:
            output_dict[value] = key
    return output_dict

def match_words(A, B):
    result = {}
    for a in A:
        words = set(a.split())
        words = [w for w in words if len(w) > 2 and w not in ["City","Town","United","Real", "West"]]
        matching_elements = [b for b in B if any(word in b for word in words)]
        result[a] = matching_elements
    return result

def filter_function(row):
    flag = row['player'].split()[-1].split('-')[-1] in [word for part in row['runner_name'].split() for word in part.split('-')]
    return flag

def get_scores(row):
    score = fuzz.ratio(row["player"], row["runner_name"])
    return score

## 1. Map team names

In [26]:
#split event name into home away teams
betfair_df[['home_team', 'away_team']] = betfair_df['event_name'].str.split(' v ', n=1, expand=True)

#get unique teams list for fbref and betfair, unidecode names
event_df_teams = pd.Series(pd.unique(event_df[['home_team', 'away_team']].values.ravel('K'))).transform(lambda x: unidecode(x))
betfair_df_teams = pd.Series(pd.unique(betfair_df[['home_team', 'away_team']].values.ravel('K'))).transform(lambda x: unidecode(x))

In [27]:
#current mapping (key:fbref -> val:betfair)
mapping = {
    'Ajaccio': ['AC Ajaccio'],
    'Arminia': ['Arminia Bielefeld'],
    'Arouca': [],
    'Athletic Club': ['Athletic Bilbao'],
    'Aves': [],
    'Bayer Leverkusen': ['Leverkusen'],
    'Belenenses SAD': [],
    'Birmingham City': ['Birmingham'],
    'Blackburn Rovers': ['Blackburn'],
    'Boavista': [],
    'Bolton Wanderers': ['Bolton'],
    'Brighton & Hove Albion': ['Brighton'],
    'Cambuur': [],
    'Cardiff City': ['Cardiff'],
    'Casa Pia': [],
    'Charlton Athletic': ['Charlton'],
    'Chaves': [],
    'Clermont Foot': ['Clermont'],
    'Coventry City': ['Coventry'],
    'Cremonese': ['US Cremonese'],
    'De Graafschap': [],
    'Deportivo La Coruna': ['Deportivo'],
    'Derby County': ['Derby'],
    'Dusseldorf': ['Fortuna Dusseldorf'],
    'Emmen': [],
    'Estoril': [],
    'Famalicao': [],
    'Farense': [],
    'Feirense': [],
    'Fortuna Sittard': [],
    'Gil Vicente FC': [],
    'Go Ahead Eagles': [],
    'Hannover 96': ['Hannover'],
    'Heerenveen': [],
    'Hellas Verona': ['Verona'],
    'Heracles Almelo': [],
    'Hertha BSC': ['Hertha Berlin'],
    'Internazionale': ['Inter Milan'],
    'Ipswich Town': ['Ipswich'],
    'Koln': ['FC Koln'],
    'Leeds United': ['Leeds', 'Leeds Utd'],
    'Leicester City': ['Leicester'],
    'Luton Town': ['Luton'],
    'Mainz 05': ['Mainz', 'FSV Mainz 05'],
    'Manchester City': ['Man City'],
    'Manchester United': ['Man Utd'],
    'Maritimo': [],
    'Milan': ['AC Milan'],
    'Monchengladbach': [],
    'Monza': ['AC Monza'],
    'Moreirense': [],
    'NAC Breda': [],
    'NEC Nijmegen': [],
    'Nacional': [],
    'Newcastle United': ['Newcastle'],
    'Norwich City': ['Norwich'],
    'Nottingham Forest': ['Nottm Forest'],
    'PSV Eindhoven': ['PSV'],
    'Pacos de Ferreira': [],
    'Paderborn 07': ['Paderborn'],
    'Paris Saint-Germain': ['Paris St-G'],
    'Peterborough United': ['Peterborough'],
    'Portimonense': [],
    'Preston North End': ['Preston'],
    'Queens Park Rangers': ['Rangers'],
    'RKC Waalwijk': [],
    'Real Betis': ['Betis'],
    'Rio Ave': [],
    'Rotherham United': ['Rotherham'],
    'Saint-Etienne': ['St Etienne'],
    'Santa Clara': [],
    'Sheffield United': ['Sheff Utd'],
    'Sheffield Wednesday': ['Sheff Wed'],
    'Sparta Rotterdam': [],
    'Sporting CP': ['Sporting Lisbon'],
    'Swansea City': ['Swansea'],
    'Tondela': [],
    'Tottenham Hotspur': ['Tottenham'],
    'Troyes': ['ESTAC Troyes'],
    'VVV-Venlo': [],
    'Vitoria Guimaraes': ['Guimaraes'],
    'Vitoria Setubal': [],
    'Vizela': [],
    'Volendam': [],
    'West Bromwich Albion': ['West Brom'],
    'West Ham United': ['West Ham'],
    'Wigan Athletic': ['Wigan'],
    'Willem II': [],
    'Wolverhampton Wanderers': ['Wolves'],
    'Wycombe Wanderers': ['Wycombe']
}

team_mapping = transform_dict(mapping)
betfair_df_teams = betfair_df_teams.transform(lambda x: team_mapping[x] if x in team_mapping.keys() else x)

In [28]:
intersection = np.intersect1d(event_df_teams, betfair_df_teams)
outersection = np.setdiff1d(event_df_teams, betfair_df_teams)
assert(len(outersection) + len(intersection) == len(event_df_teams))

print(outersection)

['Arouca' 'Aves' 'Belenenses SAD' 'Boavista' 'Cambuur' 'Casa Pia' 'Chaves'
 'De Graafschap' 'Emmen' 'Estoril' 'Famalicao' 'Farense' 'Feirense'
 'Fortuna Sittard' 'Gil Vicente FC' 'Go Ahead Eagles' 'Heerenveen'
 'Heracles Almelo' 'Maritimo' 'Monchengladbach' 'Moreirense' 'NAC Breda'
 'NEC Nijmegen' 'Nacional' 'Pacos de Ferreira' 'Portimonense'
 'RKC Waalwijk' 'Rio Ave' 'Santa Clara' 'Sparta Rotterdam' 'Tondela'
 'VVV-Venlo' 'Vitoria Setubal' 'Vizela' 'Volendam' 'Willem II']


In [29]:
#automatically search for team name pairs that share a word (see match_words function def), if raw_map contains key, val pairs,
#check if they are correct and update the mapping above and re run.
raw_map = match_words(outersection, betfair_df_teams)
raw_map

{'Arouca': [],
 'Aves': [],
 'Belenenses SAD': [],
 'Boavista': [],
 'Cambuur': [],
 'Casa Pia': [],
 'Chaves': [],
 'De Graafschap': [],
 'Emmen': [],
 'Estoril': [],
 'Famalicao': [],
 'Farense': [],
 'Feirense': [],
 'Fortuna Sittard': [],
 'Gil Vicente FC': ['Gillingham', 'Union St Gilloise'],
 'Go Ahead Eagles': [],
 'Heerenveen': [],
 'Heracles Almelo': [],
 'Maritimo': [],
 'Monchengladbach': [],
 'Moreirense': [],
 'NAC Breda': [],
 'NEC Nijmegen': [],
 'Nacional': [],
 'Pacos de Ferreira': [],
 'Portimonense': [],
 'RKC Waalwijk': [],
 'Rio Ave': [],
 'Santa Clara': [],
 'Sparta Rotterdam': ['Spartak Moscow', 'Sparta Prague'],
 'Tondela': [],
 'VVV-Venlo': [],
 'Vitoria Setubal': ['Vitoria Guimaraes'],
 'Vizela': [],
 'Volendam': [],
 'Willem II': []}

In [30]:
betfair_df_teams[betfair_df_teams.str.contains("Setu")]

Series([], dtype: object)

## 2. Map fbref and betfair matches and players

### 2.1 Matches

In [31]:
#apply pre-processing to event_df and betfair_df (unidecode and map betfair names to fbref names)
event_df['home_team'] = event_df['home_team'].transform(lambda x: unidecode(x))
event_df['away_team'] = event_df['away_team'].transform(lambda x: unidecode(x))

betfair_df['home_team'] = betfair_df['home_team'].transform(lambda x: unidecode(x))
betfair_df['away_team'] = betfair_df['away_team'].transform(lambda x: unidecode(x))

betfair_df['home_team'] = betfair_df['home_team'].transform(lambda x: team_mapping[x] if x in team_mapping.keys() else x)
betfair_df['away_team'] = betfair_df['away_team'].transform(lambda x: team_mapping[x] if x in team_mapping.keys() else x)

#rename betfair open_date as datetime, and convert both to string datetimes,remove timezone from utc betfair datetime
betfair_df["datetime"] = betfair_df.open_date.copy()
betfair_df.datetime = pd.to_datetime(betfair_df.datetime).dt.tz_localize(None)
event_df.datetime = pd.to_datetime(event_df.datetime)

#create date columns
betfair_df["date"] = pd.to_datetime(betfair_df.datetime.dt.date)
event_df["date"] = pd.to_datetime(event_df.datetime.dt.date)

#merge matches on matchdate, home_team and away_team (left merge)
betfair_matches = betfair_df[["event_name", "date", "home_team", "away_team"]].drop_duplicates()
event_matches = event_df[['home_team', 'away_team', 'date','league_name','season']].drop_duplicates()

#temporary pre-2023 and post-2017 filter!!
event_matches = event_matches[(event_matches.date < '2023-01-01') & (event_matches.date > '2018-01-01')]

matches = pd.merge(event_matches, betfair_matches, on=['date', 'home_team', 'away_team'], how='left')

### 2.2 Players

In [32]:
#repeat the merge but this time include the players
#merge matches on matchdate, home_team and away_team (left merge)
betfair_players = betfair_df[["date", "home_team", "away_team", "event_id","runner_id", "runner_name"]].drop_duplicates()
event_players = event_df[['home_team', 'away_team', 'date',"player_id", "player", "position_mapped"]].drop_duplicates()

betfair_players.runner_name = betfair_players.runner_name.transform(lambda x: unidecode(x))
event_players.player = event_players.player.transform(lambda x: unidecode(x))

#temporary pre-2023 and post-2017 filter!!
event_players = event_players[(event_players.date < '2023-01-01') & (event_players.date > '2018-01-01')]

#use only matches that exist both in betfair and fbref datasets (from 2.1)
matched_matches = matches[~matches.event_name.isnull()]
matched_players_ev = pd.merge(matched_matches, event_players, on=['date', 'home_team', 'away_team'], how='left')
matched_players = pd.merge(matched_players_ev, betfair_players, on=['date', 'home_team', 'away_team'], how='left')

In [33]:
#filter for rows where fbref player last name is one of the names in runner_name 
filtered_df = matched_players[matched_players.apply(lambda row: filter_function(row), axis=1)].copy(deep=True)

#score each row for similarity between player and runner_name
filtered_df["str_scores"] = filtered_df.apply(get_scores, axis=1)

#check duplicates
print(f"N. duplicates = {len(filtered_df[filtered_df[['home_team', 'away_team', 'date', 'player']].duplicated(keep=False)])}")

#rank rows by similarity and for each "player" keep only the best match
indices =  filtered_df.groupby(["home_team", "away_team", "date", "player"])['str_scores'].idxmax()
filtered_df = filtered_df.loc[indices]

#check duplicates
print(f"N. duplicates after removal = {len(filtered_df[filtered_df[['home_team', 'away_team', 'date', 'player']].duplicated(keep=False)])}")

N. duplicates = 1626
N. duplicates after removal = 0


## 3. Check missing values

### 3.1 Check missing matches

In [34]:
print(len(matches))
print(len(matches[matches.event_name.isnull()]))

14037
7914


In [35]:
matches['year'] = matches['date'].dt.year

# Group by year and count
result = matches.groupby('year').agg(
    total_rows=pd.NamedAgg(column='date', aggfunc='size'),
    null_event_rows=pd.NamedAgg(column='event_name', aggfunc=lambda x: x.isnull().sum())
)

display(result.head(10))

# Group by leaue and count
result = matches.groupby('league_name').agg(
    total_rows=pd.NamedAgg(column='date', aggfunc='size'),
    null_event_rows=pd.NamedAgg(column='event_name', aggfunc=lambda x: x.isnull().sum())
)

display(result.head(10))

# Group by leaue and year, and count
result = matches.groupby(['league_name','year']).agg(
    total_rows=pd.NamedAgg(column='date', aggfunc='size'),
    null_event_rows=pd.NamedAgg(column='event_name', aggfunc=lambda x: x.isnull().sum())
)

display(result.head(80))

,total_rows,null_event_rows
year,,
2018,2382,1524
2019,3005,1994
2020,2562,1509
2021,3259,1629
2022,2829,1258


,total_rows,null_event_rows
league_name,,
Bundesliga,1511,616
Championship,2525,2181
Eredivisie,1291,1287
La Liga,1881,553
Ligue 1,1769,1113
Premier League,1850,140
Primeira Liga,1349,1349
Serie A,1861,675


total_rows  null_event_rows
league_name    year                             
Bundesliga     2018         306              226
               2019         306              188
               2020         270              110
               2021         342               61
               2022         287               31
Championship   2018         300              257
               2019         555              473
               2020         518              432
               2021         578              499
               2022         574              520
Eredivisie     2018         153              150
               2019         321              320
               2020         196              196
               2021         345              345
               2022         276              276
La Liga        2018         380              152
               2019         391              177
               2020         355              177
               2021         407               39
               2022         348                8
Ligue 1        2018         369              268
               2019         388              284
               2020         260              173
               2021         400              290
               2022         352               98
Premier League 2018         366              114
               2019         379               26
               2020         336                0
               2021         408                0
               2022         361                0
Primeira Liga  2018         126              126
               2019         306              306
               2020         279              279
               2021         348              348
               2022         290              290
Serie A        2018         382              231
               2019         359              220
               2020         348              142
               2021         431               47
               2022         341               35

In [36]:
matches[(matches.league_name == "La Liga") & (matches.event_name.isnull())].head(3)

,home_team,away_team,date,league_name,season,event_name,year
193,Valencia,Girona,2018-01-06,La Liga,2017-2018,NaN,2018
196,Leganes,Real Sociedad,2018-01-07,La Liga,2017-2018,NaN,2018
198,Villarreal,Deportivo La Coruna,2018-01-07,La Liga,2017-2018,NaN,2018


### 3.2 Check missing players (for matched matches)

In [37]:
#check missing values
print(matched_players_ev.shape)
print(filtered_df.shape)

all_matches = matched_players_ev.merge(filtered_df,
                         on = ["home_team","away_team","date","league_name","season","event_name","player_id","player", "position_mapped"],
                         how='left')

# Group by year and count
result = all_matches.groupby('season').agg(
    total_rows=pd.NamedAgg(column='date', aggfunc='size'),
    null_event_rows=pd.NamedAgg(column='runner_name', aggfunc=lambda x: x.isnull().sum())
)

display(result.head(10))

# Group by leaue and count
result = all_matches.groupby('position_mapped').agg(
    total_rows=pd.NamedAgg(column='date', aggfunc='size'),
    null_event_rows=pd.NamedAgg(column='runner_name', aggfunc=lambda x: x.isnull().sum())
)

display(result.head(15))

(178645, 9)
(69426, 13)


,total_rows,null_event_rows
season,,
2017-2018,10390,5884
2018-2019,28676,16937
2019-2020,27799,13895
2020-2021,42469,27767
2021-2022,46270,30026
2022-2023,23041,14710


,total_rows,null_event_rows
position_mapped,,
AM,9566,5340
CB,31387,18035
CM,42487,27485
DM,9494,6157
FB,22223,14633
FW,26955,11707
GK,12376,12305
W,17336,8849
WB,6821,4708


# Export data

In [44]:
event_df_mapped = event_df.merge(filtered_df[["home_team", "away_team", "date","player_id", "event_id", "runner_id"]],
                                 on=["home_team", "away_team", "date","player_id"],
                                 how='left')

#un-decode team_names
event_df_mapped["home_team"] = fbref_df["home_team"]
event_df_mapped["away_team"] = fbref_df["away_team"]

#select only key columns
event_df_mapped = event_df_mapped[["home_team","away_team","datetime","player_id","event_id","runner_id"]]

In [49]:
event_df_mapped.to_csv("betfairhistoricalprices/fbref_betfair_mapping.csv", index=False)

In [50]:
event_df_mapped

,home_team,away_team,datetime,player_id,event_id,runner_id
0,Juventus,Cagliari,2017-08-19 16:00:00,8a08491a,NaN,NaN
1,Juventus,Cagliari,2017-08-19 16:00:00,c0cddebe,NaN,NaN
2,Juventus,Cagliari,2017-08-19 16:00:00,afa0d4e0,NaN,NaN
3,Juventus,Cagliari,2017-08-19 16:00:00,0ab1f153,NaN,NaN
4,Juventus,Cagliari,2017-08-19 16:00:00,4fa7cb9b,NaN,NaN
...,...,...,...,...,...,...
483153,Brentford,Manchester City,2023-05-28 15:30:00,eaeca114,NaN,NaN
483154,Brentford,Manchester City,2023-05-28 15:30:00,5cc3ce65,NaN,NaN
483155,Brentford,Manchester City,2023-05-28 15:30:00,119b9a8e,NaN,NaN
483156,Brentford,Manchester City,2023-05-28 15:30:00,86dd77d1,NaN,NaN
